In [77]:
import requests
import pandas as pd
from math import radians, sin, cos, sqrt, atan2

def calculate_distance(lat1, lon1, lat2, lon2):
    # Convert degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = 6371 * c  # Radius of Earth in kilometers

    return distance

def get_atms(api_key, latitude, longitude, radius):
    url = "https://search-maps.yandex.ru/v1/"
    params = {
        "text": "банкомат Сбербанка",
        "ll": f"{longitude},{latitude}",
        "spn": f"{radius / 111.32},{radius / (111.32 * cos(radians(latitude)))}",  # Convert 1 km to degrees
        "type": "biz",
        "apikey": api_key,
        "lang": "ru_RU",
        "results": 1000  # Increase the number of results
    }

    response = requests.get(url, params=params)
    print("Response status code:", response.status_code)
    data = response.json()
    print("Response data:", data)

    atms = []
    if "features" in data:
        for feature in data["features"]:
            name = feature["properties"]["name"]
            address = feature["properties"]["CompanyMetaData"]["address"]
            hours = feature["properties"]["CompanyMetaData"].get("Hours", "Unknown")
            atm_lat = feature["geometry"]["coordinates"][1]
            atm_lon = feature["geometry"]["coordinates"][0]
            distance = calculate_distance(latitude, longitude, atm_lat, atm_lon)
            if distance <= radius / 1000:  # Convert radius from meters to kilometers
                atms.append({"name": name, "address": address, "hours": hours})
    
    return atms

api_key = "5cccf681-14f9-4917-b12c-94f3642ee9c6"

latitude = 56.307227
longitude = 43.996895
radius = 10000  # in meters

# Getting information about ATMs
atms = get_atms(api_key, latitude, longitude, radius)
if atms:
    df = pd.DataFrame(atms)
    print(df)
    df.to_excel('банкоматы3.xlsx', index=False)
    print("Data successfully saved")
else:
    print("No ATMs found in the specified radius.")


Response status code: 200
Response data: {'type': 'FeatureCollection', 'properties': {'ResponseMetaData': {'SearchResponse': {'found': 412, 'display': 'multiple', 'boundedBy': [[42.2377359, 53.13888809], [45.0421511, 59.33218191]]}, 'SearchRequest': {'request': 'банкомат Сбербанка', 'skip': 0, 'results': 500, 'boundedBy': [[-0.91866375, -73.54941173], [88.91245375, 88.46469087]]}}}, 'features': [{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [43.99116, 56.312674]}, 'properties': {'name': 'Сбербанк', 'description': 'ул. Костина, 2, Нижний Новгород, Россия', 'boundedBy': [[43.987122, 56.310113], [43.995332, 56.314677]], 'uri': 'ymapsbm1://org?oid=185684177673', 'CompanyMetaData': {'id': '185684177673', 'name': 'Сбербанк', 'address': 'Нижний Новгород, улица Костина, 2', 'url': 'https://www.sberbank.ru/', 'Phones': [{'type': 'phone', 'formatted': '8 (800) 555-55-50'}, {'type': 'phone', 'formatted': '900'}, {'type': 'phone', 'formatted': '+7 (495) 500-55-50'}], 'Categories